# Shallow quantum circuits

In [1]:
import numpy as np

## Utility functions

In [2]:
def crandn(size):
    """
    Draw random samples from the standard complex normal (Gaussian) distribution.
    """
    # 1/sqrt(2) is a normalization factor
    return (np.random.normal(size=size) + 1j*np.random.normal(size=size)) / np.sqrt(2)

In [3]:
def retained_bond_indices(s, tol):
    """
    Indices of retained singular values based on given tolerance.
    """
    w = np.linalg.norm(s)
    if w == 0:
        return np.array([], dtype=int)
    # normalized squares
    s = (s / w)**2
    # accumulate values from smallest to largest
    sort_idx = np.argsort(s)
    s[sort_idx] = np.cumsum(s[sort_idx])
    return np.where(s > tol)[0]

In [4]:
def split_matrix_svd(A, tol):
    """
    Split a matrix by singular value decomposition,
    and truncate small singular values based on tolerance.
    """
    assert A.ndim == 2
    u, s, v = np.linalg.svd(A, full_matrices=False)
    # truncate small singular values
    idx = retained_bond_indices(s, tol)
    u = u[:, idx]
    v = v[idx, :]
    s = s[idx]
    return u, s, v

## MPS class and related utility functions

In [5]:
def local_orthonormalize_left_qr(A, Anext):
    """
    Left-orthonormalize a MPS tensor `A` by a QR decomposition,
    and update tensor at next site.
    """
    # perform QR decomposition and replace A by reshaped Q matrix
    s = A.shape
    assert len(s) == 3
    Q, R = np.linalg.qr(np.reshape(A, (s[0]*s[1], s[2])), mode='reduced')
    A = np.reshape(Q, (s[0], s[1], Q.shape[1]))
    # update Anext tensor: multiply with R from left
    Anext = np.transpose(np.tensordot(R, Anext, (1, 1)), (1, 0, 2))
    return A, Anext

In [6]:
def local_orthonormalize_right_qr(A, Aprev):
    """
    Right-orthonormalize a MPS tensor `A` by a QR decomposition,
    and update tensor at previous site.
    """
    # flip left and right virtual bond dimensions
    A = np.transpose(A, (0, 2, 1))
    # perform QR decomposition and replace A by reshaped Q matrix
    s = A.shape
    assert len(s) == 3
    Q, R = np.linalg.qr(np.reshape(A, (s[0]*s[1], s[2])), mode='reduced')
    A = np.transpose(np.reshape(Q, (s[0], s[1], Q.shape[1])), (0, 2, 1))
    # update Aprev tensor: multiply with R from right
    Aprev = np.tensordot(Aprev, R, (2, 1))
    return A, Aprev

In [7]:
def merge_mps_tensor_pair(A0, A1):
    """
    Merge two neighboring MPS tensors.
    """
    A = np.tensordot(A0, A1, (2, 1))
    # pair original physical dimensions of A0 and A1
    A = A.transpose((0, 2, 1, 3))
    # combine original physical dimensions
    A = A.reshape((A.shape[0]*A.shape[1], A.shape[2], A.shape[3]))
    return A

In [8]:
class MPS(object):
    """
    Matrix product state (MPS) class.

    The i-th MPS tensor has dimension `[d, D[i], D[i+1]]` with `d` the physical
    dimension at each site and `D` the list of virtual bond dimensions.
    """

    def __init__(self, d, D, fill='zero'):
        """
        Create a matrix product state.
        """
        self.d = d
        # leading and trailing bond dimensions must agree (typically 1)
        assert D[0] == D[-1]
        if fill == 'zero':
            self.A = [np.zeros((d, D[i], D[i+1])) for i in range(len(D)-1)]
        elif fill == 'random real':
            # random real entries
            self.A = [np.random.normal(size=(d, D[i], D[i+1])) / np.sqrt(d*D[i]*D[i+1]) for i in range(len(D)-1)]
        elif fill == 'random complex':
            # random complex entries
            self.A = [crandn(size=(d, D[i], D[i+1])) / np.sqrt(d*D[i]*D[i+1]) for i in range(len(D)-1)]
        else:
            raise ValueError('fill = {} invalid.'.format(fill))

    @classmethod
    def from_tensors(cls, Alist):
        """
        Construct a MPS from a list of tensors.
        """
        # create a MPS with dummy tensors
        s = cls(2, (len(Alist) + 1) * [1])
        # assign the actual tensors from `Alist`
        s.A = [np.array(A) for A in Alist]
        s.d = s.A[0].shape[0]
        # consistency checks
        for j in range(len(s.A)):
            assert s.A[j].ndim == 3, "Each MPS tensor must be of degree 3."
            assert s.d == s.A[j].shape[0], "Physical dimension not consistent across MPS tensors."
            assert s.A[j].shape[2] == s.A[(j+1) % len(s.A)].shape[1], "Incompatible virtual bond dimensions."
        return s

    @property
    def local_dim(self):
        """Local (physical) dimension at each lattice site."""
        return self.d

    @property
    def nsites(self):
        """Number of lattice sites."""
        return len(self.A)

    @property
    def bond_dims(self):
        """Virtual bond dimensions."""
        if len(self.A) == 0:
            return []
        else:
            D = [self.A[i].shape[1] for i in range(len(self.A))]
            D.append(self.A[-1].shape[2])
            return D

    @property
    def dtype(self):
        """Data type of tensor entries."""
        return self.A[0].dtype

    def orthonormalize(self, mode='left'):
        """Left- or right-orthonormalize the MPS using QR decompositions."""
        if len(self.A) == 0:
            return

        if mode == 'left':
            for i in range(len(self.A) - 1):
                self.A[i], self.A[i+1] = local_orthonormalize_left_qr(self.A[i], self.A[i+1])
            # last tensor
            self.A[-1], T = local_orthonormalize_left_qr(self.A[-1], np.array([[[1.0]]]))
            # normalization factor (real-valued since diagonal of R matrix is real)
            assert T.shape == (1, 1, 1)
            nrm = T[0, 0, 0].real
            if nrm < 0:
                # flip sign such that normalization factor is always non-negative
                self.A[-1] = -self.A[-1]
                nrm = -nrm
            return nrm
        elif mode == 'right':
            for i in reversed(range(1, len(self.A))):
                self.A[i], self.A[i-1] = local_orthonormalize_right_qr(self.A[i], self.A[i-1])
            # first tensor
            self.A[0], T = local_orthonormalize_right_qr(self.A[0], np.array([[[1.0]]]))
            # normalization factor (real-valued since diagonal of R matrix is real)
            assert T.shape == (1, 1, 1)
            nrm = T[0, 0, 0].real
            if nrm < 0:
                # flip sign such that normalization factor is always non-negative
                self.A[0] = -self.A[0]
                nrm = -nrm
            return nrm
        else:
            raise ValueError('mode = {} invalid; must be "left" or "right".'.format(mode))

    def as_vector(self):
        """Merge all tensors to obtain the vector representation on the full Hilbert space."""
        psi = self.A[0]
        for i in range(1, len(self.A)):
            psi = merge_mps_tensor_pair(psi, self.A[i])
        assert psi.ndim == 3
        # contract leftmost and rightmost virtual bond (has no influence if these virtual bond dimensions are 1)
        psi = np.trace(psi, axis1=1, axis2=2)
        return psi

In [9]:
def split_mps_tensor(A, d0, d1, svd_distr, tol=0):
    """
    Split a MPS tensor with dimension `d0*d1 x D0 x D2` into two MPS tensors
    with dimensions `d0 x D0 x D1` and `d1 x D1 x D2`, respectively.
    """
    assert A.ndim == 3
    assert d0 * d1 == A.shape[0], 'physical dimension of MPS tensor must be equal to d0 * d1'
    # reshape as matrix and split by SVD
    A = np.transpose(np.reshape(A, (d0, d1, A.shape[1], A.shape[2])), (0, 2, 1, 3))
    s = A.shape
    A0, sigma, A1 = split_matrix_svd(A.reshape((s[0]*s[1], s[2]*s[3])), tol)
    A0.shape = (s[0], s[1], len(sigma))
    A1.shape = (len(sigma), s[2], s[3])
    # use broadcasting to distribute singular values
    if svd_distr == 'left':
        A0 = A0 * sigma
    elif svd_distr == 'right':
        A1 = A1 * sigma[:, None, None]
    elif svd_distr == 'sqrt':
        s = np.sqrt(sigma)
        A0 = A0 * s
        A1 = A1 * s[:, None, None]
    else:
        raise ValueError('svd_distr parameter must be "left", "right" or "sqrt".')
    # move physical dimension to the front
    A1 = A1.transpose((1, 0, 2))
    return A0, A1

In [10]:
def is_left_orthonormal(A):
    """
    Test whether a MPS tensor `A` is left-orthonormal.
    """
    s = A.shape
    assert len(s) == 3
    A = np.reshape(A, (s[0]*s[1], s[2]))
    return np.allclose(A.conj().T @ A, np.identity(s[2]))

In [11]:
def is_right_orthonormal(A):
    """
    Test whether a MPS tensor `A` is right-orthonormal.
    """
    # call `is_left_orthonormal` with flipped left and right virtual bond dimensions
    return is_left_orthonormal(np.transpose(A, (0, 2, 1)))

## Shifted sequence of two-qubit gates

In [12]:
def shifted_gate_sequence_to_mps(Ulist):
    """
    Represent the output of a quantum circuit consisting of
    a shifted sequence of two-qubit gates as MPS.

             ____
     |0> ---|    |------------------------------
            | U1 |    ____
     |0> ---|____|---|    |---------------------
                     | U2 |                   .
     |0> ------------|____|---                .
                               ...      ____  .
                                    ---|    |---
                                       | U* |
     |0> ------------------------------|____|---

    """
    Alist = []
    for U in Ulist:
        # application to |0> selects entries corresponding to index 0 at last leg
        A = np.reshape(U, (2, 2, 2, 2))[:, :, :, 0]
        # still have to flip virtual bond dimensions
        A = np.transpose(A, (0, 2, 1))
        Alist.append(A)
    # contract with topmost qubit in |0> state, keeping a (dummy) virtual bond dimension 1
    Alist[0] = Alist[0][:, 0:1, :]
    # last MPS tensor acts as identity
    Alist.append(np.reshape(np.identity(2), (2, 2, 1)))
    return MPS.from_tensors(Alist)

### Example: sequence of CNOT gates

In [13]:
# Hadamard gate
H = np.array([[1., 1.], [1., -1.]]) / np.sqrt(2)
H

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [14]:
# CNOT gate
Ucnot = np.identity(4)[[0, 1, 3, 2]]
Ucnot

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [15]:
d = 5
Ulist = [Ucnot @ np.kron(H, np.identity(2)) if j == 0 else Ucnot for j in range(d-1)]

If the topmost qubit starts in state $|1\rangle$ (and the remaining ones in $|0\rangle$), then the sequence of CNOTs
flips all the remaining qubits one-by-one, arriving at $|11 \dots 1\rangle$.

Here we first apply the Hadamard gate to the topmost qubit, which generates the state $(|0\rangle + |1\rangle)/\sqrt{2}$.
By the superposition principle, the final output is then $(|00 \dots 0\rangle + |11 \dots 1\rangle)/\sqrt{2}$.
In vector form, precisely the first and last entry should be equal to $1/\sqrt{2}$, and all other entries zero.

In [16]:
ψ = shifted_gate_sequence_to_mps(Ulist)

In [17]:
# agrees with analytical prediction
ψ.as_vector()

array([0.70710678, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.70710678])

### Example: sequence of random gates (for additional testing)

In [18]:
# for reference calculation
def apply_two_qubit_gate(psi, d, U, i, j):
    """
    Apply the two-qubit gate `U` (acting on qubits i, j) to state vector `psi`.
    `d` is the overall number of qubits.
    """
    assert 0 <= i < j < d
    assert len(psi) == 2**d
    # isolate dimensions corresponding to the qubits the gate acts on
    psi = np.reshape(psi, (2**i, 2, 2**(j-i-1), 2, 2**(d-j-1)))
    # reshape gate into a tensor of degree 4
    U = np.reshape(U, (2, 2, 2, 2))
    # actually apply gate (last argument determines dimension ordering of output)
    psi = np.einsum(U, (1, 5, 2, 4), psi, (0, 2, 3, 4, 6), (0, 1, 3, 5, 6))
    # flatten back to a vector
    psi = np.reshape(psi, -1)
    return psi

In [19]:
d = 5
# random unitary gates
Ulist = [np.linalg.qr(crandn((4, 4)))[0] for j in range(d-1)]

In [20]:
# reference calculation
# start with unit vector corresponding to |00...0>
χ = np.zeros(2**d)
χ[0] = 1
# apply gates
for j in range(d-1):
    χ = apply_two_qubit_gate(χ, d, Ulist[j], j, j+1)

In [21]:
# should still be normalized
np.linalg.norm(χ)

1.0000000000000002

In [22]:
# show entries
χ

array([ 0.01853535+0.12058069j,  0.13202861+0.09829634j,
       -0.10457343-0.10939952j, -0.03467736+0.10282173j,
       -0.01224346-0.09890856j,  0.05603962-0.01977798j,
       -0.02209161+0.08445907j,  0.0152397 +0.01529514j,
       -0.07239654-0.12229871j, -0.05168653+0.05658178j,
        0.06411507+0.03624088j, -0.04961934-0.03812987j,
       -0.0196241 -0.04382513j,  0.10406197-0.07667183j,
       -0.0341942 +0.09121832j,  0.05270061+0.04287218j,
       -0.03111464+0.0009412j , -0.10043367+0.05370974j,
        0.06797315-0.04955231j, -0.04693685-0.04910051j,
        0.09737139+0.00767138j,  0.01008283+0.10203335j,
       -0.08603514-0.06302986j, -0.04226432+0.018331j  ,
       -0.06134561-0.43091114j, -0.2382391 +0.15692401j,
        0.14322066+0.13986859j, -0.13105216-0.17328179j,
        0.04109717-0.18737146j,  0.44235284-0.05005634j,
       -0.27450025+0.22773645j,  0.08677888+0.21506166j])

In [23]:
ψ = shifted_gate_sequence_to_mps(Ulist)

In [24]:
# compare
np.allclose(ψ.as_vector(), χ)

True

## Shifted double sequence of two-qubit gates

In [25]:
def shifted_double_gate_sequence_to_mps(Ulist, Vlist):
    """
    Represent the output of a quantum circuit consisting of
    two shifted sequences of two-qubit gates as MPS.

             ____              ____
     |0> ---|    |------------|    |------------------------------------------------
            | U1 |    ____    | V1 |    ____
     |0> ---|____|---|    |---|____|---|    |---------------------------------------
                     | U2 |    ____    | V2 |
     |0> ------------|____|---|    |---|____|---                                  .
                              | U3 |                                              .
     |0> ---------------------|____|---        .                                  .
                                                   .               ____
                                                       .       ---|    |------------
                                                          ____    | V* |    ____
                                                      ---|    |---|____|---|    |---
                                                         | U* |            | V* |   
     |0> ------------------------------------------------|____|------------|____|---

    """
    assert len(Ulist) == len(Vlist)
    # convert sequence of U gates to a MPS
    psi = shifted_gate_sequence_to_mps(Ulist)
    # apply the V gates to the MPS (similar to the TEBD algorithm)
    for i in range(len(Vlist)):
        Apair = merge_mps_tensor_pair(psi.A[i], psi.A[i+1])
        assert Apair.ndim == 3
        # apply V[i]
        Apair = np.tensordot(Vlist[i], Apair, axes=(1, 0))
        # set "tolerance" to zero to avoid truncation errors
        psi.A[i], psi.A[i+1] = split_mps_tensor(Apair, psi.local_dim, psi.local_dim, "right", tol=0)
    return psi

### Example: sequence of random gates (for testing)

In [26]:
d = 5
# random unitary gates
Ulist = [np.linalg.qr(crandn((4, 4)))[0] for j in range(d-1)]
Vlist = [np.linalg.qr(crandn((4, 4)))[0] for j in range(d-1)]

In [27]:
# reference calculation
# start with unit vector corresponding to |00...0>
χ = np.zeros(2**d)
χ[0] = 1
# apply gates
for j in range(d-1):
    χ = apply_two_qubit_gate(χ, d, Ulist[j], j, j+1)
for j in range(d-1):
    χ = apply_two_qubit_gate(χ, d, Vlist[j], j, j+1)

In [28]:
# should still be normalized
np.linalg.norm(χ)

1.0

In [29]:
ψ = shifted_double_gate_sequence_to_mps(Ulist, Vlist)

In [30]:
# compare
np.allclose(ψ.as_vector(), χ)

True